# Semantic Kernel Tool Use Example

This document provides an overview and explanation of the code used to create a Semantic Kernel-based tool that integrates with ChromaDB for Retrieval-Augmented Generation (RAG). The example demonstrates how to build an AI agent that retrieves travel documents from a ChromaDB collection, augments user queries with semantic search results, and streams detailed travel recommendations.

## Initializing the Environment

In [2]:
import os
import asyncio
import chromadb

from openai import AsyncOpenAI
from semantic_kernel.kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.contents import ChatHistory

A Semantic Kernel instance is then created, and the OpenAI chat completion service is added to it.

In [3]:
# Initialize the asynchronous OpenAI client
client = AsyncOpenAI(
    api_key=os.environ["GITHUB_TOKEN"],
    base_url="https://models.inference.ai.azure.com/"
)

# Create a Semantic Kernel instance and add an OpenAI chat completion service.
kernel = Kernel()
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
    service_id="agent",
)
kernel.add_service(chat_completion_service)

## Creating the Agent

In the **Creating the Agent** section, the code defines:
- `AGENT_NAME`: The name of the AI agent, e.g., `"TravelAgent"`.
- `AGENT_INSTRUCTIONS`: A string that instructs the agent on its task, tone, and behavior (helpful, travel recommendation focused).

The agent is instantiated using `ChatCompletionAgent`, which utilizes the Semantic Kernel and the previously defined service.

In [4]:
AGENT_NAME = "TravelAgent"
AGENT_INSTRUCTIONS = (
    "You are a helpful AI Agent that can help plan vacations for customers. "
    "When formulating your response, base your answer solely on the information provided in the search results below; do not include external knowledge."
)
agent = ChatCompletionAgent(service_id="agent", kernel=kernel, name=AGENT_NAME)

## Setting Up ChromaDB

To facilitate Retrieval-Augmented Generation:
- A persistent ChromaDB client is instantiated.
- A collection called `"travel_documents"` is created (or retrieved if it exists). This collection contains sample travel documents and metadata.
- Sample documents describing various travel services (e.g., luxury vacation packages, itinerary planning, travel insurance) are added to the collection.

In [5]:
# Initialize ChromaDB with persistent storage
chroma_client = chromadb.PersistentClient(path="./chroma_db")
collection = chroma_client.create_collection(
    name="travel_documents",
    metadata={"description": "travel_service"},
    get_or_create=True
)

# Enhanced sample documents
documents = [
    "Contoso Travel offers luxury vacation packages to exotic destinations worldwide.",
    "Our premium travel services include personalized itinerary planning and 24/7 concierge support.",
    "Contoso's travel insurance covers medical emergencies, trip cancellations, and lost baggage.",
    "Popular destinations include the Maldives, Swiss Alps, and African safaris.",
    "Contoso Travel provides exclusive access to boutique hotels and private guided tours."
]

# Add documents with metadata
collection.add(
    documents=documents,
    ids=[f"doc_{i}" for i in range(len(documents))],
    metadatas=[{"source": "training", "type": "explanation"} for _ in documents]
)

## Text Search Plugin

The **TextSearchPlugin** class is implemented to interface with ChromaDB:
- The `search` method performs a query on the collection using a provided search term. It returns the first relevant document along with its corresponding metadata formatted as a string.


In [6]:
class TextSearchPlugin:
    """
    A text search plugin that uses ChromaDB to retrieve travel documents.
    """
    def __init__(self, chroma_db_path: str = "./chroma_db", collection_name: str = "travel_documents"):
        self.client = chromadb.PersistentClient(path=chroma_db_path)
        try:
            self.collection = self.client.get_collection(name=collection_name)
        except Exception as e:
            raise Exception(
                f"Collection '{collection_name}' not found."
            ) from e

    def search(self, query: str) -> str:
        """
        Searches the ChromaDB collection for documents relevant to the query.
        Returns a formatted string with the first search result's document and metadata.
        """
        results = self.collection.query(
            query_texts=[query],
            include=["documents", "metadatas"],
            n_results=1
        )
        if results and results.get("documents") and len(results["documents"][0]) > 0:
            result_text = results["documents"][0][0]  # First result document text.
            metadata = results["metadatas"][0][0]       # Corresponding metadata.
            return f"Document: {result_text}\nMetadata: {metadata}"
        return "No results found"

## Augmenting User Queries

The function `semantic_search_rag` is defined to augment user queries with semantic search results from ChromaDB. It uses the `TextSearchPlugin` to retrieve relevant documents based on the user's input. The function then formats the search results as a string and appends them to the user's query.

In [7]:
def semantic_search_rag(query: str) -> str:
    # Initialize the text search plugin (uses default ChromaDB path and collection "travel_documents")
    search_plugin = TextSearchPlugin()
    search_result = search_plugin.search(query)
    
    # Construct an augmented prompt containing both the search results and the original query.
    prompt = (
        f"Search Results:\n{search_result}\n\n"
        f"User Query: {query}\n\n"
        "Based solely on the above search results, provide a detailed travel recommendation using only the provided information."
    )
    return prompt

# Running the Agent

The agent is run in a loop that prompts the user for input and provides travel recommendations based on the user's queries. The agent uses the `semantic_search_rag` function to augment the user's queries with semantic search results from ChromaDB.

In [8]:
async def main():
    # Initialize chat history and set system instructions.
    chat_history = ChatHistory()
    chat_history.add_system_message(AGENT_INSTRUCTIONS)

    # List of user queries.
    user_inputs = [
        "What travel services does Contoso offer?",
        "Are there any luxury vacation options?"
    ]

    for user_input in user_inputs:
        # Print the user's original message.
        print(f"User: {user_input}")
        
        # Create augmented prompt using semantic search.
        augmented_prompt = semantic_search_rag(user_input)
        chat_history.add_user_message(augmented_prompt)
        
        # Stream the agent's response asynchronously.
        response_text = ""
        async for content in agent.invoke_stream(chat_history):
            if content.content.strip():
                response_text += content.content
        
        # Print the assistant's response.
        print(f"Assistant: {response_text}\n")

In [9]:
if __name__ == "__main__":
    if asyncio.get_event_loop().is_running():
        # Running in an interactive environment, use await main()
        await main()
    else:
        # Running in a standard script, use asyncio.run()
        asyncio.run(main())

User: What travel services does Contoso offer?
Assistant: Contoso Travel specializes in luxury vacation packages to exotic destinations worldwide. If you're looking for a top-tier travel experience, I recommend exploring some of their offerings that may include stunning resorts, unique cultural experiences, and serene environments.To enhance your vacation, consider planning visits to destinations that are known for their luxury accommodations and beautiful scenery. Activities may include relaxing spa treatments, gourmet dining, and guided tours of local attractions.For a truly memorable experience, look into specific packages that Contoso Travel offers, focusing on exotic locations that intrigue you. Since they cater to luxury travel, you're likely to find exceptional services tailored to meet your needs throughout your vacation.Overall, if you're seeking an unforgettable getaway, Contoso Travel's luxury vacation packages would be an excellent choice for indulging in a dream vacation.
